In [1]:
import os

In [2]:
%pwd

'd:\\recommendation-engine\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\recommendation-engine'

In [5]:
import pandas as pd

In [6]:
data = pd.read_csv("artifacts/data_ingestion/generated_data/bookings.csv")
data.head()

,user_id,booked_event_id,booked_event_organizer_id,booking_id
0,ee00034d-050a-4d08-b82f-f73234e36f18,49632a17-c8de-4dc1-b70e-1d82df4365b8,1a469fec-484c-483c-8e9b-c8d7d379ddbd,241d563b-9aa5-443c-ac20-d34e34beab68
1,baba8c29-dbb6-4fdf-82c3-033ccaee01cd,5056d7d9-7561-4878-add6-80d1f7d7dc14,cf6095dc-decb-4331-8cc1-9c52952d095b,5623eef6-3273-49a0-9758-7746ca620995
2,936a5038-8683-4b10-a3b7-0c35e8f968b4,ea404f23-cfc1-4563-8de0-321bfc003f0d,515858ee-fbff-48c3-841c-c717ddb54701,ae2e687f-624a-484e-8ee6-d78409c2c582
3,1e720e97-ce89-4c4f-90ef-a2651ae06f58,a5d876e7-d61a-46a9-8b3a-f8de70e15004,5aae934f-5a2d-4e38-a0f4-54a3c174d2a2,710d59f9-d49c-4f42-8379-ca543e033d77
4,baed69ca-149e-465f-a21e-0b4e9e124899,d490e063-72bc-4a36-a87f-120c4db28a3f,9e396395-0952-497c-9a01-01598c0f8f1c,e7cae6ee-2ee8-4ada-8b9c-ffa7897874c1


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   user_id                    100000 non-null  object
 1   booked_event_id            100000 non-null  object
 2   booked_event_organizer_id  100000 non-null  object
 3   booking_id                 100000 non-null  object
dtypes: object(4)
memory usage: 3.1+ MB


In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    merged_data_url: Path
    all_schema: dict

In [9]:
from src.hybrid_recommender.constants import *
from src.hybrid_recommender.utils.common import read_yaml, create_directories

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            merged_data_url = config.merged_data_url,
            all_schema=schema,
        )

        return data_validation_config

In [11]:
import os
from src.hybrid_recommender import logger


In [12]:
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.merged_data_url)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e

In [13]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2025-06-17 05:45:25,206: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-17 05:45:25,210: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-17 05:45:25,220: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-06-17 05:45:25,220: INFO: common: created directory at: artifacts]
[2025-06-17 05:45:25,220: INFO: common: created directory at: artifacts/data_validation]
